In [6]:
#load aiida environment and connect to database
from aiida import load_profile
from aiida.orm import computers
profile = load_profile()
print('profile name : ',profile.name)
print('profile database host name : ',profile.database_hostname)
print('default user : ',profile.default_user)

profile name :  default
profile database host name :  localhost
default user :  r.mozumder@fz-juelich.de


In [7]:
# load classes and functions
from aiida.orm import CifData, Code, Dict, load_node, StructureData 
from aiida.orm import (Group, load_group, load_node, load_code, groups,
                      WorkChainNode, QueryBuilder)
from aiida.engine import submit
from aiida.common.exceptions import NotExistent
from aiida_kkr.workflows import kkr_imp_sub_wc, kkr_imp_dos, kkr_imp_wc, kkr_startpot_wc
import numpy as np
from aiida_kkr.calculations import KkrimpCalculation, VoronoiCalculation
import matplotlib.pyplot as plt
from aiida_kkr.tools.common_workfunctions import get_username

In [8]:
# Structure data from voronoi calc or kkr_startpot_wc
def Struc_from_voro(voro_node, host_structure=True, imp_structure=False):
    # Packages and module
    from aiida.common.exceptions import InputValidationError
    from aiida_kkr.calculations import VoronoiCalculation
    from aiida_kkr.workflows import kkr_startpot_wc 
    from aiida.orm import StructureData, Node
    """
    Structure from the voronoi calcjob or voro_startpot_wc
    voro_node: (voro_calc)
    """
    # To check voro_node in node or ID
    if not isinstance(voro_node, Node):
        try:
            voro_node = load_node(voro_node)
        except IndentationError:
            print('No such pk or uuid: {} is present in the database.'.format(voro_node))
            
    host_struc, imp_host_struc = None, None
    
    # For voronoi calcjob
    if (voro_node.process_class==VoronoiCalculation) :
        ## Degug here
        print('This Voronoicalculation')
        struc = voro_node.inputs.structure
        # Check for is it from ancector wc or created there
        # For example the struc from sub kkr_startpot_wc in kkr_imp_wc is created there so 
        # no inconing_node is available 
        
        if struc.get_incoming().first() == []:
            imp_host_struc = struc
    # For kkr_startpot_wc  
        
    

In [10]:
## Trying to develop to delete the node from database
## and at the same time from the remote folder

## Add the following possible services
# 1. Give the print option for how many decendant node will be 
#       and take the permission. Add option wether need to take 
#         permission or not.
# 2. Also print how many calcjob node will be deleted under permission 
# 3. Print'remote directory' to check that all the data from the
#     remote dir are deleted or not.
## Technique
# 1. Use the QuaryDB()
# Use the cleandir



#Section-1: delete node from the database as well as remote work directory
### A function to delete the data of calculation output of calcjob list (pks) from the remote dir.

# later add it with the del_node  function
# This is successfully done

# please note that it is tested for one calc list
def delete_remote_workdir(pks, verbosity=0, dry_run= True):
    from aiida.common import exceptions
    from aiida.orm import load_node
    from aiida.orm import computers
    import sys
    # TODO : add the verbosity as discused here 
    # https://aiida.readthedocs.io/projects/aiida-core/en/v1.5.0/_modules/aiida/manage/database/delete/nodes.html

    """
    :param pks: calc node list
    
    """
    removed_path_list = [] # The part of the path will be deleted
    remote_path_list = []  # The original path
    updated_path_list = [] # After removing the part of the path
    loadable_list = [] # To load the node and save it loadable_list
    loaded_node_list = []
    # To check the loadable calcjob list
    for pk in pks:
        try:
            loaded_node = load_node(pk)
        except exceptions.NotExistent:
            print('This is calcjob node'.format(pk))
            loaded_node = pk
            loaded_node_list.append(loaded_node)
            #             sys.exit()
        else:
            loaded_node_list.append(loaded_node)
    # Computer data
    
   
    for node in loaded_node_list:
        load_pk = node
        # computer data
        computer = load_pk.computer
        computer_name = computer.label
        print(computer_name)
        
        remote_path = load_pk.get_remote_workdir()
        remote_path_list.append(remote_path)
        
        delete_folder = remote_path.split('/')[-1]
        removed_path_list.append(delete_folder)

        new_remote_path = remote_path.replace(remote_path.split('/')[-1], '')
        updated_path_list.append(new_remote_path)
        
        
        
    if dry_run or verbosity==3:

        for i, paths in enumerate(zip(remote_path_list, updated_path_list)):
            print('Before the delation the original path list : {}\n'.format(paths[0]))
            print('After deletion the modefied or updated path : {}'.format(paths[1]))
    val = input("Are you agree to clean the remote workdir (y/n) : ")
    
    if str(val)=='y' or str(val)=='Y':
        for remote_path in remote_path_list:
            if not dry_run:
                    # Open the connection to the remote folder/dir via transport
                    computer_transport = computer.get_transport()
                    is_transport_open = computer_transport.is_open
                    if not is_transport_open:
                        computer_transport.open()
                    try:
                        computer_transport.rmtree(remote_path)
                    except IOError as ex:
                        print(ex)
    else:
        print('Nothing to clean from the remote workdir!')
# section-5:delete node from the database as well as remote work directory
## It returns all the calcjob from a WC node
def find_calcJob(pk_or_node, debug=True):
    
    calcjob_node_list=[]
    wc_node_list = []
    try:
        if isinstance( pk_or_node, int):
            if debug:
                print('This is pk')
            node = load_node(pk_or_node)
        else:
            if debug:
                print('This is node.')
            node= pk_or_node
    except:
        print('{} is nither node ID nor aiida_node. '.format(pk_or_node))
        
    ## Use the get_calcjob_wc to get descendent calcjob list and  wc list
    calc_list, wc_list = get_calcjob_wc(node)
    calcjob_node_list += calc_list
    
    while len(wc_list)!=0:
        new_wc_list = []

        for i in wc_list[:]:
            calc_list, wc_list = get_calcjob_wc(i)
            new_wc_list += wc_list
            calcjob_node_list += calc_list
            
        wc_list = new_wc_list

    return calcjob_node_list

## This function returns calcjob_list and wc_list from a wc or calcjob node   
def get_calcjob_wc(node):
    """
    :param: node
    :return: workchain node list and calcjob node list
    """ 
    from aiida.orm import CalcJobNode, WorkChainNode
    wc = []
    calc_job = []
    
    if node.node_type == 'process.workflow.workchain.WorkChainNode.':
        
    # here all outgoing worchain node
        out_going_wc = node.get_outgoing(node_class=WorkChainNode).all()
        wc = [i.node for i in out_going_wc[:]]
        
    # here all outgoing calcjob node
        out_going_calc = node.get_outgoing(node_class=CalcJobNode).all()
        calc_job = [i.node for i in out_going_calc[:]]
                    
    elif node.node_type == 'process.calculation.calcjob.CalcJobNode.':
        calc_job.append(node)
    
    return calc_job, wc

# This is the final del_node_function. Using this function for any specific wc node the node from the 
# Db as well as the calcjob data from the remote workdir can be deleted.
def del_nodes(node_pks, dry_run=True, verbosity=3, debug=True, only_workdir= False,
             only_database= False):
    """
    1. This function will delete the node data from the database and also from the remote_dir
    
    :params node_pks: (list) list of workchain to delete from database as well as from remote workdir
    :param verbosity: 0 prints nothing.  This is for workdir and wc
                      1 prints just sums and total.   This is for workdir but not for wc
                      2 prints indivisual nodes.  This is for workdir and wc
    :param dry_run: Do not delete anything just show the status as in the verbosity given
    """
    from aiida.orm import load_node
    from aiida.manage.database.delete.nodes import delete_nodes
        
    calcjobs_list = []
    pks_given = []
    for i in node_pks:    
        try:
            if isinstance( i, int):
                if debug:
                    print('This might be pk or uiid')
                node = load_node(i)
            else:
                if debug:
                    print('This might be a node.')
                node= i
        except:
            print('{} is nither node ID nor aiida_node. '.format(i))
        
        pks_given.append(node.pk)
        
        calcjobs = find_calcJob(node, debug)
        calcjobs_list += calcjobs
        print('calcjob list : ', calcjobs_list,)
    if only_workdir:
        delete_remote_workdir(calcjobs_list, verbosity=verbosity, dry_run=dry_run)
    if only_database:
        delete_nodes(pks_given, verbosity=verbosity, dry_run=dry_run,force=False)
    

In [20]:
group_107 = load_group(105)
node_list = list(group_107.nodes)
del_nodes(node_pks=node_list, dry_run= False, verbosity=3, debug= False, 
          only_workdir= True, only_database= True)

calcjob list :  [<CalcJobNode: uuid: 335db72e-5890-4707-a2f0-ff4869c4698f (pk: 40851) (aiida.calculations:kkr.kkrimp)>]
calcjob list :  [<CalcJobNode: uuid: 335db72e-5890-4707-a2f0-ff4869c4698f (pk: 40851) (aiida.calculations:kkr.kkrimp)>, <CalcJobNode: uuid: c765c1b9-aa04-4e72-9b90-454e0c12af9c (pk: 41635) (aiida.calculations:kkr.kkrimp)>]
calcjob list :  [<CalcJobNode: uuid: 335db72e-5890-4707-a2f0-ff4869c4698f (pk: 40851) (aiida.calculations:kkr.kkrimp)>, <CalcJobNode: uuid: c765c1b9-aa04-4e72-9b90-454e0c12af9c (pk: 41635) (aiida.calculations:kkr.kkrimp)>, <CalcJobNode: uuid: f570d959-eb89-40bf-a591-aa06bb915468 (pk: 40876) (aiida.calculations:kkr.kkrimp)>]
calcjob list :  [<CalcJobNode: uuid: 335db72e-5890-4707-a2f0-ff4869c4698f (pk: 40851) (aiida.calculations:kkr.kkrimp)>, <CalcJobNode: uuid: c765c1b9-aa04-4e72-9b90-454e0c12af9c (pk: 41635) (aiida.calculations:kkr.kkrimp)>, <CalcJobNode: uuid: f570d959-eb89-40bf-a591-aa06bb915468 (pk: 40876) (aiida.calculations:kkr.kkrimp)>, <Calc

Are you agree to clean the remote workdir (y/n) :  y


I will delete 570 nodes
The nodes I will delete:
   566a5927-6f81-4c1a-955e-5a603d6bc93e 40594 WorkChainNode Sc:Fe:Bi2Te3_il_3_il_3
   272a2f35-f9b0-4933-9bfd-8cc66e55b5b9 40597 WorkChainNode Sc:Co:Bi2Te3_il_3_il_3
   667837f4-5406-4c8e-901d-613052aab2cf 40631 CalcFunctionNode create_combined_imp_info_cf
   fa9649e5-493e-4fc1-84c6-585e6e45c2e8 40635 Dict 
   9b53a7d7-3aab-4573-a9eb-9e6f17636a44 40636 Dict 
   e502e968-8a10-4ce1-9ccd-5c175452c71b 40641 CalcFunctionNode combine_potentials_cf
   14da98c8-a75d-4241-9d66-af3edf879282 40646 SinglefileData combined_potentials
   8662e989-686e-466c-bfa9-6bb6581389c5 40648 CalcFunctionNode create_combined_imp_info_cf
   84865f09-9837-4822-930a-b5ff5d1d6575 40651 WorkChainNode kkrimp scf combined imps
   cb0e609a-f2fe-421a-84ee-9f91eede8fcf 40654 Dict 
   4dc1cc90-4e2c-43ea-9caa-d391806b5ca6 40656 Dict 
   24125864-63ce-4aa4-8a2e-2e327422841a 40664 CalcFunctionNode combine_potentials_cf
   61b5d1db-6d1b-47a5-9c2f-0b731519b919 40667 SinglefileDat

 y


Starting node deletion...
Nodes deleted from database, deleting files from the repository now...
Deletion completed.


[<CalcJobNode: uuid: 4f96b1e1-7bf9-442e-8a50-5147b733e794 (pk: 24667) (aiida.calculations:kkr.kkrimp)>,
 <CalcJobNode: uuid: b2b07931-8919-4216-ab1d-179be5f02979 (pk: 24635) (aiida.calculations:kkr.kkr)>,
 <CalcJobNode: uuid: 8cbbabd5-8288-469b-a045-9569e8a14876 (pk: 24650) (aiida.calculations:kkr.kkrimp)>,
 <CalcJobNode: uuid: 79ae3a10-9c5d-4b40-b44b-37bb294c9f81 (pk: 24657) (aiida.calculations:kkr.kkrimp)>]

claix18_init
claix18_init
Before the delation the original path list : /rwthfs/rz/cluster/work/jara0191/ck142666/aiida/computers/claix18_init/8c/bb/abd5-8288-469b-a045-9569e8a14876

After deletion the modefied or updated path : /rwthfs/rz/cluster/work/jara0191/ck142666/aiida/computers/claix18_init/8c/bb/
Before the delation the original path list : /rwthfs/rz/cluster/work/jara0191/ck142666/aiida/computers/claix18_init/79/ae/3a10-9c5d-4b40-b44b-37bb294c9f81

After deletion the modefied or updated path : /rwthfs/rz/cluster/work/jara0191/ck142666/aiida/computers/claix18_init/79/ae/


Are you agree to clean the remote workdir (y/n) :  n


Nothing to clean from the remote workdir!


In [33]:
#section-2: delete node from the database as well as remote work directory
## This code  will help to workchain as explained in the filter
qb = QueryBuilder()
node_list = list(qb.append([WorkChainNode],
                    filters={
                        'and':[
                            {'attributes.process_label':'combine_imps_wc'},
                            {'or':[
                                {'attributes.process_status': 'excepted'},
                                {'attributes.exit_status':{'!in':[0]}},
                                {'attributes.process_status': 'killed'},
                                
                                 ]
                            }
                        ]
                    }
                   
                   
                   ).all())

# node_list

In [35]:
## section-3 : delete node from the database as well as remote work directory
## To check any specific pk in the pk_list exist or not
def check_pk_exist(pk_list, pk):
    for i in pk:
        for j in pk_list:
            if(i==j):
                print('pk-{} is exist in pk_list'.format(i))
                continue

len(node_list)

pk_list = [i[0].pk for i in node_list]
pk_list.sort()
print(check_pk_exist(pk_list,[22232]))

None
